<a href="https://colab.research.google.com/github/aswann2/219/blob/main/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Compare all season wins, goal scored to and against, age of team, percent of games won(seasons may be different lengths), power play goals, Location(Northern teams v. more southern teams?), beginning of season predictions(Canes have been projected to win and they will win(speaking this into existence)), Division(We can group them into the current way that divisions are set up and go from there), penalties in minutes per game,

'23 winners=Vegas Golden Knights

'22 winners=Colorado Avalanche

'21 Winners=Tampa Bay lightning

'20 winners=Lightning

'19 winners= St. Louis Blues

- AvAge,

- W=wins

- L=losses

- GF, =goals for

- GA, = goals against

- SOW, = Shootout wins

- SOL= Shootout losses

- Create column for shootouts?,

- GF/G= goals for per game

- GA/G= goals against per game

- PP= power play goals

- PPO= power play opportunities

- SH= Short handed goals

- PIM/G= penalties in minutes per game

- S=Shots on goal

- S%= shooting percentage

- SO= shutouts

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression

In [ ]:
# Create an empty list that will later store all 5 year team statistic dataframes
seasons = []

# Read in the CSV files into a dataframe
S22_23= pd.read_csv('https://raw.githubusercontent.com/aswann2/219/main/S22-23.csv')
S21_22= pd.read_csv('https://raw.githubusercontent.com/aswann2/219/main/S21-22.csv')
S20_21= pd.read_csv('https://raw.githubusercontent.com/aswann2/219/main/S20-21.csv')
S19_20= pd.read_csv('https://raw.githubusercontent.com/aswann2/219/main/S19-20.csv')
S18_19= pd.read_csv('https://raw.githubusercontent.com/aswann2/219/main/S18-19.csv')

# Get the team rows of those who won the Stanley Cup
season_winners = pd.concat([S22_23.loc[S22_23['Team'] == 'Vegas Golden Knights*'],
                            S21_22.loc[S21_22['Team'] == 'Colorado Avalanche*'],
                            S20_21.loc[S20_21['Team'] == 'Tampa Bay Lightning*'],
                            S19_20.loc[S19_20['Team'] == 'Tampa Bay Lightning*'],
                            S18_19.loc[S18_19['Team'] == 'St. Louis Blues*']],
                            ignore_index=True)

# Create another column in each data frame and set the value to 1 for those who won the Stanley Cup
S22_23['won_stanley_cup'] = 0
S22_23.loc[S22_23['Team'] == 'Vegas Golden Knights*', 'won_stanley_cup'] = 1

S21_22['won_stanley_cup'] = 0
S21_22.loc[S21_22['Team'] == 'Colorado Avalanche*', 'won_stanley_cup'] = 1

S20_21['won_stanley_cup'] = 0
S20_21.loc[S20_21['Team'] == 'Tampa Bay Lightning*', 'won_stanley_cup'] = 1

S19_20['won_stanley_cup'] = 0
S19_20.loc[S19_20['Team'] == 'Tampa Bay Lightning*', 'won_stanley_cup'] = 1

S18_19['won_stanley_cup'] = 0
S18_19.loc[S18_19['Team'] == 'St. Louis Blues*', 'won_stanley_cup'] = 1

# Add all dataframes into the list
seasons.append(S22_23)
seasons.append(S21_22)
seasons.append(S20_21)
seasons.append(S19_20)
seasons.append(S18_19)

# Combine all data into one dataframe
allSeasons = pd.concat(seasons)
allSeasons

,Rk,Team,AvAge,GP,W,L,OL,PTS,PTS%,GF,...,SH,SHA,PIM/G,oPIM/G,S,S%,SA,SV%,SO,won_stanley_cup
0,1.0,Boston Bruins*,29.5,82,65,12,5,135,0.823,301,...,8,8,10.5,9.9,2703,11.1,2447,0.929,7,0
1,2.0,Carolina Hurricanes*,29.3,82,52,21,9,113,0.689,262,...,11,6,7.5,7.5,2852,9.2,2132,0.902,9,0
2,3.0,New Jersey Devils*,26.7,82,52,22,8,112,0.683,289,...,9,8,7.7,7.9,2821,10.2,2315,0.904,4,0
3,4.0,Vegas Golden Knights*,29.1,82,51,22,9,111,0.677,267,...,10,4,7.2,7.9,2587,10.3,2533,0.911,3,1
4,5.0,Toronto Maple Leafs*,28.6,82,50,21,11,111,0.677,278,...,8,8,8.4,8.4,2628,10.6,2420,0.909,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,28.0,Detroit Red Wings,29.3,82,32,40,10,74,0.451,224,...,7,7,7.9,7.3,2394,9.4,2761,0.901,2,0
28,29.0,New Jersey Devils,27.5,82,31,41,10,72,0.439,219,...,10,10,9.1,9.0,2488,8.8,2592,0.895,6,0
29,30.0,Los Angeles Kings,29.3,82,31,42,9,71,0.433,199,...,4,13,8.1,8.0,2358,8.4,2578,0.900,5,0
30,31.0,Ottawa Senators,27.3,82,29,47,6,64,0.390,242,...,4,8,8.5,8.3,2429,10.0,2931,0.897,4,0


In [36]:
features= ['AvAge','W','L','GF','GA','SOW','SOL','GF/G','GA/G','PP','PPO','SH','PIM/G','S','S%','SO']

#Pipeline
pipeline=make_pipeline(
    StandardScaler(),
    KNeighborsRegressor(n_neighbors=5)
)

X_train = allSeasons[features].fillna(0)
Y_train = allSeasons['won_stanley_cup']

pipeline.fit(X_train, Y_train)
Y_train_predict= pipeline.predict(X_train)
mse = mean_squared_error(Y_train, Y_train_predict)
rmse = np.sqrt(mse)

print('MSE: ' + str(mse))
print('RMSE: ' + str(rmse))

MSE: 0.02641975308641976
RMSE: 0.16254154264808662
